#### Section 1.4.4 Linear Regulator
Taken from "Switch-Mode Power Supplies" by Christophe P. Basso

#### Imports

In [1]:
from pathlib import Path
import py4spice as spi
import copy

#### Path to Ngspice and the project
***IMPORTANT:*** These variables must be filled in by the user.\
The notebook calculates the rest of the paths from these.

In [2]:
# Path to NGSPICE
NGSPICE_EXE = Path("/usr/bin/ngspice")

# Path to KiCad
KICAD_EXE = Path("/usr/bin/kicad")
## add to devcontainer: sudo apt install kicad kicad-libraries &&

# Absolute path to my project
PROJ_PATH = Path("/workspaces/sw_pwr_book_sim/circuits/sec_1_04_04_lin_reg")

# Name of dut (device under test) schematic
DUT_SCHEMATIC_NAME = "sec_1_04_04_lin_reg.kicad_sch"

#### Prepare other paths

In [3]:
# Create sim_results directory if doesn't exist
RESULTS_PATH = PROJ_PATH / "sim_results"
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

# Path to schematic
SCHEMATIC_PATH = PROJ_PATH / "kicad_schematics"

# Create netlists directory and files if they don't exist
NETLISTS_PATH = PROJ_PATH / "netlists"
NETLISTS_PATH.mkdir(parents=True, exist_ok=True)  # Create the directory

#### Create dut.cir file from Kicad schematic

In [4]:
DUT_SCH_FILENAME: Path = SCHEMATIC_PATH / DUT_SCHEMATIC_NAME
KICAD_RAW_FILENAME = SCHEMATIC_PATH / "raw_kicad.cir"

#### Define netlists

In [5]:

# Create title.cir with content only if it doesn't exist
TITLE_FILENAME = NETLISTS_PATH / "title.cir"
if not TITLE_FILENAME.exists():
    with open(TITLE_FILENAME, "w") as f:
        f.write("* First line in netlist must be a comment")
title = spi.Netlist(NETLISTS_PATH / TITLE_FILENAME)

# These netlist objects will be combined and a top netlist will be created later
load1 = spi.Netlist(NETLISTS_PATH / "load_resistive.cir")
load2 = spi.Netlist(NETLISTS_PATH / "load_resistive.cir")
load3 = spi.Netlist(NETLISTS_PATH / "load_current_pulse.cir")
stimulus1 = spi.Netlist(NETLISTS_PATH / "stimulus_15v_dc.cir")
stimulus2 = spi.Netlist(NETLISTS_PATH / "stimulus_15v_ramp.cir")
stimulus3 = spi.Netlist(NETLISTS_PATH / "stimulus_15v_dc.cir")
supplies = spi.Netlist(NETLISTS_PATH / "supplies.cir")
models = spi.Netlist(NETLISTS_PATH / "models.cir")

# Create "end" netlist object, but no need to write to a file
end = spi.Netlist()
end.insert_line(0, ".end")

# two files we'll create later for first simulation
TOP1_FILENAME = NETLISTS_PATH / "top1.cir"
CONTROL1_FILENAME = NETLISTS_PATH / "control1.cir"

# two files we'll create later for second simulation
TOP2_FILENAME = NETLISTS_PATH / "top2.cir"
CONTROL2_FILENAME = NETLISTS_PATH / "control2.cir"

# two files we'll create later for third simulation
TOP3_FILENAME = NETLISTS_PATH / "top3.cir"
CONTROL3_FILENAME = NETLISTS_PATH / "control3.cir"

# Create dut.cir from raw_kicad.cir
raw_kicad = spi.Netlist(NETLISTS_PATH / "raw_kicad.cir")
raw_kicad.del_line_starts_with(".title")  # delete first line (title)
raw_kicad.del_line_starts_with(".end")  # delete last line (.end)
raw_kicad.del_line_starts_with(".include")  # delete first  .include line
raw_kicad.del_line_starts_with(".include")  # delete second .include line
raw_kicad.del_slash()   # delete forward slashes from node names
dut = raw_kicad
dut.write_to_file(NETLISTS_PATH / "dut.cir")  # this is now the dut (device under test)

#### Create vectors

In [6]:
VEC_ALL = spi.Vectors("all")    # "all" gives us the default signals from a simulation

### Simulation 1
Operating point analysis and small signal transfer function

#### Define analyses

In [7]:
list_of_analyses1: list[spi.Analyses] = [] # create empty list. Next sections define

# 1st analysis: operating point
op_cmd = "op"
op1 = spi.Analyses("op1", "op", op_cmd, VEC_ALL, RESULTS_PATH)
list_of_analyses1.append(op1)

# 2nd analysis: transfer function
tf_cmd = "tf v(out) vin"
tf1 = spi.Analyses("tf1", "tf", tf_cmd, VEC_ALL, RESULTS_PATH)
list_of_analyses1.append(tf1)

# # 3rd analysis: transient
# tr_cmd = "tran 1e-9 20e-6"
# tr1 = spi.Analyses("tr1", "tran", tr_cmd, VEC_ALL, RESULTS_PATH)
# list_of_analyses.append(tr1)

#### Create control section

In [8]:
my_control1 = spi.Control()  # create 'my_control' object
my_control1.insert_lines(["listing"])  # cmd to list out netlist
for analysis in list_of_analyses1:  # statements for all analyses
    my_control1.insert_lines(analysis.lines_for_cntl())
my_control1.content_to_file(CONTROL1_FILENAME)  # create the actual file
control1 = spi.Netlist(NETLISTS_PATH / CONTROL1_FILENAME)  # create netlist object
spi.print_section("Control File", my_control1)  # print out contents


--- Control File ---
.control
* Timestamp: Sat Mar  2 20:49:35 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line all > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_04_lin_reg/sim_results/op1.txt
tf v(out) vin
print line all > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_04_lin_reg/sim_results/tf1.txt
quit
.endc
--------------------



#### Simulate

In [9]:
# Combine the netlists and write out into one top netlist, ready to simulate
top1 = title + dut + load1 + supplies + stimulus1 + models + control1 + end

top1.write_to_file(TOP1_FILENAME)
spi.print_section("top netlist", top1)

# prepare simulate object, print out command, and simulate
sim1 = spi.Simulate(NGSPICE_EXE, TOP1_FILENAME)
spi.print_section("Ngspice Command", sim1)
sim1.run() # run the Ngspice simulation


--- top netlist ---
* linear regulator, section 1.4.4
rf1 beta rc 100
cf1 rc div 100n
rlower1 div com 10k
e1 beta com vref div 100
rupper1 out div 10k
e2 g com beta com 10
x2 in gain gain k=0.05
rsol1 sum out 1
x3 gain g sum sum k1=1 k2=1 a=1
rload out out_meas 50
vmeas out_meas 0 dc 0
vcom com 0 dc 0
vref vref 0 dc 2.5
vin in 0 dc 15
.subckt gain in out k=1
e1 out 0 in 0 {k}
.ends gain

.subckt sum in1 in2 out k1=1 k2=1 a=1
b1 out 0 v = {a * ((v(in1) * k1) + (v(in2) * k2))}
.ends sum
.control
* timestamp: sat mar  2 20:49:35 2024
set wr_singlescale  $ makes one x-axis for wrdata
set wr_vecnames     $ puts names at top of columns
listing
op
print line all > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_04_lin_reg/sim_results/op1.txt
tf v(out) vin
print line all > /workspaces/sw_pwr_book_sim/circuits/sec_1_04_04_lin_reg/sim_results/tf1.txt
quit
.endc
.end
-------------------


--- Ngspice Command ---
/usr/bin/ngspice -b /workspaces/sw_pwr_book_sim/circuits/sec_1_04_04_lin_reg/netlists/

#### Simulation results to numpy

In [10]:
# convert the raw results into list of SimResults objects
my_sim_results1 = [
    spi.SimResults.from_file(analysis.cmd_type, analysis.results_filename)
    for analysis in list_of_analyses1
]
# give each SimResults object a more descriptive name
op1_results, tf1_results = my_sim_results1

#### Operating point values from op1 results

In [11]:
spi.print_section("Operating Point Results", op1_results.print_table())


--- Operating Point Results ---
b.x3.b1#branch  -0.100076
beta             0.4341393
com              0.0
div              2.495659
e.x2.e1#branch   0.0
e1#branch        0.0
e2#branch        0.0
g                4.341393
gain             0.75
in               15.0
out              4.991317
out_meas         0.0
rc               0.4341393
sum              5.091393
vcom#branch      0.0002495659
vin#branch       0.0
vmeas#branch     0.09982634
vref             2.5
vref#branch      0.0
-------------------------------



#### Small signal transfer function results from tf1 results

In [12]:
spi.print_section("Transfer Function Results", tf1_results.print_table())


--- Transfer Function Results ---
transfer_function            9.979641e-05
output_impedance_at_v(out)   0.001995928
vin#input_impedance          1e+20
---------------------------------



### Simulation 2
Transient with Vin ramp

#### Results from small signal DC transfer function analysis

In [13]:
print("hello world")

hello world
